In [12]:
import os
import time
import logging
from typing import Dict, List
from functools import lru_cache

import httpx
import pandas as pd

In [2]:
token_endpoint = str(os.getenv("TOKEN_ENDPOINT"))
api_host = str(os.getenv("API_HOST"))
api_endpoint = str(os.getenv("API_ENDPOINT"))
proxy_endpoint = str(os.getenv("FORD_PROXY"))
scope = str(os.getenv("SCOPE"))
client_id = str(os.getenv("CLIENT_ID"))
client_secret = str(os.getenv("CLIENT_SECRET"))

os.environ["HTTP_PROXY"] = proxy_endpoint
os.environ["HTTPS_PROXY"] = proxy_endpoint
os.environ["NO_PROXY"] = api_host

In [13]:
def create_client() -> httpx.Client:
    """
    Creates a common client for future http requests

    Returns:
        httpx.Client: client with ford proxies
    """
    ford_proxy = str(os.getenv("FORD_PROXY"))
    timeout_config = httpx.Timeout(10.0, connect=5.0)
    proxy_mounts = {
        "http://": httpx.HTTPTransport(proxy=httpx.Proxy(ford_proxy)),
        "https://": httpx.HTTPTransport(proxy=httpx.Proxy(ford_proxy)),
    }
    return httpx.Client(
        timeout=timeout_config,
        mounts=proxy_mounts,
        verify=False,
    )

In [21]:
def get_token():
    with create_client() as client:
        response = client.post(
            str(os.getenv("TOKEN_ENDPOINT")),
            data={
                "client_id": str(os.getenv("CLIENT_ID")),
                "client_secret": str(os.getenv("CLIENT_SECRET")),
                "scope": str(os.getenv("SCOPE")),
                "grant_type": "client_credentials",
            },
            timeout=160,
        )
    print(
        "token expires in:",
        round(int(response.json()["expires_in"]) / 60, 0),
        "minutes",
    )
    return response.json()["access_token"]


def load_categories():
    categories = []
    with open("./binnings.txt", encoding="utf-8", mode="r") as file:
        for line in file.readlines():
            categories.append(line.strip("\n"))

    return frozenset(categories)


token = get_token()

token expires in: 60.0 minutes


In [15]:
@lru_cache(maxsize=70)
def call_api_movement(token, complaint, categories):
    parts = (
        "door, window, windshield, wiper, glass, hood, trunk, moonroof, "
        + "bumper, tail light, pillar, undershield, roof rack, latch, he"
        + "adlight, door handle, door keypad, window, weatherstripping, "
        + "side mirror, lighting, swing gate, cowl grille, hard top, ski"
        + "d plate, sheet metal, running boards, water leak, etc)"
    )
    prompt = (
        f"{complaint}"
        "Question 1: For this complaint, check if it is related to an ex"
        + f"ternal part of the car, body exterior, ({parts}). If ye"
        + "s, answer 'F8'. Otherwise, answer 'NOT F8'. Note that most of"
        + "the problems related to power liftgate electrical problems an"
        + "d rear view camera are NOT F8. Question 2: For each of these "
        + "sentences that your answer 1 was 'F8', check if it is related"
        + f"to only one of the following categories: {list(categories)}."
        + " You should give only one answer with one answer for Question"
        + " 1 and one answer for Question 2 in the following format: 'AN"
        + "SWER 1~~~ANSWER 2'. Note: 'OWD' means 'opened while driving' "
        + "and 'F&F' means 'fit and finish', for problems related to flu"
        + "shness and margin. Note 2: For model Escape (2020 forward), t"
        + "here is a common problem related to door check arm when the c"
        + "omplaint is related to the door making popping sounds, openin"
        + "g and closing problens, hinges and welds. If you cannot relat"
        + "e, answer NOT SURE. Answer in the correct order. If you canno"
        + "t assist, answer 1, and answer 2 must be NA. You should be ob"
        + "jective and cold. Never change the answer format mentioned."
    )
    content = {
        "model": "gpt-4",
        "context": (
            "You are a helpful text reader and analyzer. You need to give me 2 answers."
        ),  # sets the overall behavior of the assistant.
        "messages": [{"role": "user", "content": prompt}],
        "parameters": {
            "temperature": 0.05,  # Determines the randomnes of the model's response.
        },
    }
    with create_client() as client:
        response = client.post(
            api_endpoint,
            headers={"Authorization": f"Bearer {token}"},
            json=content,
            timeout=360,
        )
    return response.json()["content"]

In [24]:
categories = list(load_categories())
df = pd.read_csv("./mock_complaints.csv")



for idx, row in enumerate(df.itertuples(index=False), start=2):

    decription = f"model {str(row[2])} of year {str(row[3])} {str(row[21])}"


    response = call_api_movement(token, decription, categories)  # type: ignore

    if response == "F8~~~" + row[1]:


        print(idx, "OK")


        continue
    print(response)

TypeError: unhashable type: 'list'